In [1]:
import os
import h5py

In [2]:
root_dir = os.getcwd()
filepath = os.path.join(root_dir, "mask_rcnn_prostate_pretrained.h5")

In [3]:
f = h5py.File(filepath, mode='r')

In [10]:
print(f.attrs, f.keys())
print(f.name)
for name in f:
    print(name)

<Attributes of HDF5 object at 139928697420264> KeysView(<HDF5 file "mask_rcnn_prostate_pretrained.h5" (mode r)>)
/
ROI
activation_1
activation_10
activation_11
activation_12
activation_13
activation_14
activation_15
activation_16
activation_17
activation_18
activation_19
activation_2
activation_20
activation_21
activation_22
activation_23
activation_24
activation_25
activation_26
activation_27
activation_28
activation_29
activation_3
activation_30
activation_31
activation_32
activation_33
activation_34
activation_35
activation_36
activation_37
activation_38
activation_39
activation_4
activation_40
activation_41
activation_42
activation_43
activation_44
activation_45
activation_46
activation_47
activation_48
activation_49
activation_5
activation_50
activation_51
activation_52
activation_53
activation_54
activation_55
activation_56
activation_57
activation_58
activation_59
activation_6
activation_60
activation_61
activation_62
activation_63
activation_64
activation_65
activation_66
activ

In [30]:
dataset1 = f['res5b_out']

In [31]:
for name in dataset1:
    print(name)

for att in dataset1.attrs:
    print(att)

m = dataset1.attrs['weight_names']
print(m)
# print(dataset1.name)
# dataset2 = dataset1['tumor_class_dense']
# print(dataset2)

weight_names
[]


In [24]:
for name in dataset2:
    print(name)


bias:0
kernel:0


In [25]:
for att in dataset2.attrs:
    print(att)